# 3. LangChain with DeepSeek-R1

**Goal:** Provide an understanding of the LangChain components and LangChain examples. 

### **What is LangChain?**
> LangChain is a framework for developing applications powered by language models.

LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs
2. **Agency** - Allow your LLMs to interact with it's environment via decision making. Use LLMs to help decide which action to take next

We will use [ollama](https://ollama.com/) for DeepSeek Model from local machine.

In [1]:
! pip install -q streamlit langchain-core langchain-community unstructured langchain_ollama

In [2]:
import os
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# LangChain Components



### **Chat Model**
A model that takes a series of messages and returns a message output

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [5]:
llm=ChatOllama(
    model="deepseek-r1:latest",
    base_url="http://localhost:11434",
    temperature=0.3

)

You can also pass more chat history w/ responses from the AI

In [6]:
response=llm(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)
print(response.content)

/var/folders/b4/twk1kqn144z6m04fvtpv4n2h0000gn/T/ipykernel_1475/1332848018.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=llm(


<think>
Alright, let me break this down. The user previously asked about beaches and I suggested Nice, France. Now they're asking what else to do in that location.

Hmm, maybe they're planning a short trip and want more activities beyond just the beach. They might be looking for things to do, see, or experience.

I should consider popular attractions near Nice. The Côte d'Azur is nearby, so suggesting Monaco makes sense because it's famous and close by.

Also, the user might appreciate cultural spots like the Nice Museum of Modern Art. It adds variety to their trip.

Fishing villages are popular too; they offer a different vibe and local experiences. So including that would be good.

I should keep each suggestion concise since they want one short sentence per point. Making sure it's helpful and covers different aspects of travel.
</think>

You could visit the nearby Monaco Grand Prix Circuit or explore the cultural scene at the Nice Museum of Modern Art, or take a boat tour to nearby f

You can also exclude the system message if you want

In [7]:
response=llm(
    [
        HumanMessage(content="Write an article in 2 paragraph on LangChain.")
    ]
)
print(response.content)

<think>
Okay, so I need to write a two-paragraph article about LangChain. Hmm, where do I start? Well, first, I should probably figure out what LangChain is. From what I remember, it's related to AI and machine learning. Maybe it's an open-source library or something like that.

Wait, isn't LangChain part of the Hugging Face ecosystem? Yeah, I think so. It was developed by Facebook, now Meta. So maybe it's used for building applications with large language models. Large language models are those big AI systems like GPT-3 or BERT, right?

So, the first paragraph should probably introduce LangChain as an open-source library built on top of Hugging Face Transformers. It allows developers to create custom applications using pre-trained language models. That makes sense because it abstracts some of the complexities.

Then I should mention its key features. Oh yeah, composition is a big one. So you can chain different models or components together to build more complex systems. Also, integra

## Prompts - Text generally used as instructions to your model
### **Prompt**
What you'll pass to the underlying model

In [8]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:latest")

prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

response=llm.invoke(prompt)
print(response.content)

<think>
First, I need to understand the user's question. They provided a statement: "Today is Monday, tomorrow is Wednesday." My task is to identify what is incorrect about this statement.

I know that in a week, each day follows another in sequence. Starting from Monday, the next day should be Tuesday, then Wednesday. So if today is Monday, tomorrow should logically be Tuesday.

However, the user's statement says that tomorrow is Wednesday. This creates an inconsistency because based on standard time progression, after Monday comes Tuesday, not Wednesday.

Therefore, the mistake lies in claiming that tomorrow will be Wednesday when it should actually be Tuesday.
</think>

**Solution:**

Let's analyze the given statement step by step to identify the error.

1. **Given Statement:**
   - Today is Monday.
   - Tomorrow is Wednesday.

2. **Understanding the Calendar:**
   - Days of the week follow in a sequence:
     \[
     \text{Sunday} \rightarrow \text{Monday} \rightarrow \text{Tuesday

### **Prompt Template**
An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain import PromptTemplate

from langchain_ollama import ChatOllama

llm = ChatOllama(model="deepseek-r1:latest")

# "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Delhi')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
response=llm.invoke(final_prompt)
print (f"LLM Output: {response.content}")

Final Prompt: 
I really want to travel to Delhi. What should I do there?

Respond in one short sentence

-----------
LLM Output: <think>
Okay, so the user wants to travel to Delhi and is asking for advice on what they should do there. They've asked me to respond in just one short sentence.

First, I need to figure out the most essential things someone new to Delhi would want to experience or visit. I remember that India Gate is a major landmark with beautiful architecture and significance. It's often recommended as a must-see for travelers.

Also, Red Fort comes to mind. It's another iconic site in Delhi, part of the Mughal influence. Including it makes sense because it's well-known and has a lot of historical importance.

Maybe also mention agra, but since the user asked for one sentence, I can't include too much. So perhaps just suggest visiting India Gate or Red Fort as must-see sites.

I should make sure to keep it concise and clear, so they know where to start their exploration in

### Output Parsers 

This output parser can be used when you want to return multiple fields. While the Pydantic/JSON parser is more powerful, this is useful for less powerful models.

In [10]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

In [11]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

**We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.**


In [12]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [13]:
model = ChatOllama(model="deepseek-r1:latest")
chain = prompt | model | output_parser

In [14]:
chain.invoke({"question": "What's the capital of India?"})

{'answer': 'Mumbai', 'source': 'Memory and basic knowledge of geography.'}

### **Documents**
An object that holds a piece of text and metadata (more information about that text)

In [15]:
from langchain.schema import Document

In [16]:
response=Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "The LangChain Papers",
             'my_document_create_time' : 1680013019
         })
print(response)

page_content='This is my document. It is full of text that I've gathered from other places' metadata={'my_document_id': 234234, 'my_document_source': 'The LangChain Papers', 'my_document_create_time': 1680013019}


But you don't have to include metadata if you don't want to

In [17]:
response=Document(page_content="This is my document. It is full of text that I've gathered from other places")
print(response)

page_content='This is my document. It is full of text that I've gathered from other places'


## Indexes - Structuring documents to LLMs can work with them
### **Document Loaders**
**HackerNews**

In [18]:
from langchain.document_loaders import HNLoader

loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [19]:
data = loader.load()

In [20]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman on Jan 18, 2023  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are veOzzie_osman on Jan 18, 2023  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_ind


**Books from Gutenberg Project**

In [21]:
from langchain.document_loaders import GutenbergLoader

loader = GutenbergLoader("https://www.gutenberg.org/cache/epub/2148/pg2148.txt")

data = loader.load()

In [22]:
print(data[0].page_content[1855:1984])

o.—_Seneca_.





      At Paris, just after dark one gusty evening in the autumn of 18-,


      I was enjoying the twofold l


**URLs and webpages**

Let's try it out with [Paul Graham's website](http://www.paulgraham.com/)

In [23]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = [
    "http://www.paulgraham.com/",
]

loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

data[0].page_content

'New: The Origins of Wokeness | Founder Mode Want to start a startup? Get funded by Y Combinator . © mmxxv pg'

### **Text Splitters**
Often times your document is too long (like a book) for your LLM. You need to split it up into chunks. Text splitters help with this.

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
# This is a long document we can split up.
with open('data/worked.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")

You have 1 document


In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

In [27]:
print (f"You have {len(texts)} documents")
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

You have 610 documents
Preview:
February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what 

beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,


### **Retrievers**
Easy way to combine documents with language models.

There are many different types of retrievers, the most widely supported is the VectoreStoreRetriever

In [28]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

loader = TextLoader('data/worked.txt')
documents = loader.load()

In [29]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [30]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x33bd79d60>, search_kwargs={})

In [31]:
docs = retriever.invoke("what types of things did the author want to build?")

In [32]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

standards; what was the point? No one else wanted one either, so
off they went. That was what happened to systems work.I wanted not just to build things, but to build things that would
last.In this di

I needed that seed funding to live on.We originally hoped to launch in September, but we got more ambitious
about the software as we worked on it. Eventually we managed to
build a WYSIWYG site builder


### Chat Message History

In [34]:
from langchain.memory import ChatMessageHistory
from langchain_ollama import ChatOllama

chat = ChatOllama(model="deepseek-r1:latest")

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of India?")

In [35]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of India?', additional_kwargs={}, response_metadata={})]

In [36]:
ai_response = chat(history.messages)
ai_response

AIMessage(content="<think>\nAlright, so I need to figure out what the capital of India is. Hmm, I know a few capitals off the top of my head, but India isn't one I've really paid much attention to before. Let me think... I remember that Delhi is a big city in India and it's often mentioned when talking about travel or news. Maybe Delhi is the capital? \n\nWait, I'm not 100% sure. I should try to recall if there are any other capitals close to me that I might confuse with Delhi. For example, I know that Mumbai is another major city in India, but I don't think it's the capital. Then there's Kolkata, which was the old capital a few decades ago. But does it still hold some importance? \n\nI also remember hearing about Hyderabad as an important city too. Could it be the capital now? Or maybe it hasn't changed much from before. Let me try to recall any information I might have learned in school. I think my teacher mentioned that Mumbai was once the capital, but Delhi is now considered the of

In [37]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="<think>\nAlright, so I need to figure out what the capital of India is. Hmm, I know a few capitals off the top of my head, but India isn't one I've really paid much attention to before. Let me think... I remember that Delhi is a big city in India and it's often mentioned when talking about travel or news. Maybe Delhi is the capital? \n\nWait, I'm not 100% sure. I should try to recall if there are any other capitals close to me that I might confuse with Delhi. For example, I know that Mumbai is another major city in India, but I don't think it's the capital. Then there's Kolkata, which was the old capital a few decades ago. But does it still hold some importance? \n\nI also remember hearing about Hyderabad as an important city too. Could it be the capital now? Or maybe it hasn't changed much from be

### Agents

The language model that drives decision making.

More specifically, an agent takes in an input and returns a response corresponding to an action to take along with an action input.

### Toolkit

Groups of tools that your agent can select from

Let's bring them all together:
(Register here for API https://serpapi.com/manage-api-key)

In [38]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_ollama import ChatOllama
import json

llm = ChatOllama(model="deepseek-r1:latest")

In [39]:
!export SERP_API_KEY="58baec75dbad0620f59fe693592a0677d42f05d9ad287c564aed091c3c9ac389"
serpapi_api_key=os.getenv("SERP_API_KEY", "58baec75dbad0620f59fe693592a0677d42f05d9ad287c564aed091c3c9ac389")

In [40]:
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)
response = agent({"input":"How is the whether outside in Delhi Today?"})

/var/folders/b4/twk1kqn144z6m04fvtpv4n2h0000gn/T/ipykernel_1475/810165310.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)
/var/folders/b4/twk1kqn144z6m04fvtpv4n2h0000gn/T/ipykernel_1475/810165310.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in l



> Entering new AgentExecutor chain...
<think>
Okay, so I need to find out the current weather conditions outside in Delhi today. I'm not exactly sure where to start, but I remember that weather information can be obtained through search engines or apps. Since I don't have access to specific app data here, maybe I can use a general search engine.

I should probably phrase my search query clearly so that it returns the most relevant results. Maybe something like "weather in Delhi today" would work. That's straightforward and should give me the necessary information about temperature, humidity, wind speed, and any precipitation expected.

Wait, but how reliable is this method? I've used Google Search before for weather info, and it usually provides up-to-date data. However, since the time is changing every second, the latest data might not be available immediately. But compared to other methods like relying on memory or using apps that aren't installed here, a search query seems better.

## Chains ⛓️⛓️⛓️
Combining different LLM calls and action automatically

Ex: Summary #1, Summary #2, Summary #3 > Final Summary


We'll cover two of them:

### 1. Basic  Chains

- Easy chains where you can use the prompt as an input into LLM. 
- Using Prompt template with placeholder for variable and passing variable in the end.

In [41]:
import os
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOllama(model="deepseek-r1:latest")

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "bears"})

'<think>\nOkay, so I need to come up with a joke about bears. Let me think about what\'s funny about bears first. They\'re cute but can be a bit grumpy sometimes. Maybe something involving their hibernation or something they do while awake.\n\nHmm, I remember hearing that bears are more dangerous than they look because of something called "baiting." Is that right? Baiting is when they go to places like campsites and then attack when people leave food. That could be funny if I play it up a bit.\n\nWait, let me make sure. Baiting isn\'t just about campsites; it can happen anywhere with bait like meat or berries. So maybe the joke could involve someone setting out meat in a forest and the bear attacking instead of the person.\n\nI should structure it so that when the person leaves the food, the bear comes first because it\'s more efficient. That contrast between the human leaving something behind but the animal acting on its own is funny.\n\nLet me put it together: "Why don’t bears set th

### 2. Simple Sequential Chains

Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [42]:
import warnings
from langchain_ollama import ChatOllama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain_core._api.deprecation import LangChainDeprecationWarning 
warnings.filterwarnings('ignore', category=LangChainDeprecationWarning)

llm = ChatOllama(model="deepseek-r1:latest")

In [43]:
template = """Your job is to come up with a classic dish from the area that the users suggests. Respond with Dish name and One line description of it.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [44]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [45]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [46]:
review = overall_chain.invoke("Delhi")



> Entering new SimpleSequentialChain chain...
<think>
Okay, so I need to figure out what the user wants here. They mentioned their job is to come up with a classic dish from Delhi based on what the user suggests. The user provided an example where they asked for a location (Delhi), and in response, I suggested "Aloo Gobi," which is a popular vegetable curry from Delhi.

First, I should understand the structure of the interaction. It seems like there's a flow: the user provides a location or something related, and I respond with a dish name and its description. So, in this case, after the initial prompt where they just provided "Delhi," my response was about Aloo Gobi.

But wait, looking back at the history, it looks like I might have missed something. The user's instruction says: "Your job is to come up with a classic dish from the area that the users suggests." So maybe in their turn, they would suggest an area or a cuisine, and then I respond accordingly.

In my last interaction, s

### 3. Summarization Chain

Easily run through long numerous documents and get a summary. 

In [47]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('data/disc.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# There is a lot of complexity hidden in this one line.
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"January 2017Because biographies of famous scientists tend to 
edit out their mistakes, we underestimate the 
degree of risk they were willing to take.
And because anything a famous scientist did that
wasn't a mistake has probably now become the
conventional wisdom, those choices don't
seem risky either.Biographies of Newton, for example, understandably focus
more on physics than alchemy or theology.
The impression we get is that his unerring judgment
led him straight to truths no one else had noticed.
How to explain all the time he spent on alchemy
and theology?  Well, smart people are often kind of
crazy.But maybe there is a simpler explanation. Maybe"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"the smartness and the craziness were not as separate
as we think. Physics seems to us a promis

"<think>\nAlright, let me process this step by step.\n\nThe user provided a text about Isaac Newton and his beliefs on different fields—physics, alchemy, and theology. The original query is asking for a concise summary of this text.\n\nFirst, I need to identify the main points:\n\n1. **Newton's Beliefs**: He saw physics, alchemy, and theology as interconnected but treated them differently.\n2. **Perception Before Newton**: These fields were often seen as separate—physics as science, alchemy as mystical, and theology as philosophical.\n3. **Interconnectedness**: Newton believed in a unified knowledge seeking to understand the universe, so all areas contributed to this understanding.\n4. **Bets on Fields**: He invested in three main areas: optics, mechanics, and gravitation. One (physics) succeeded; the others didn't.\n\nNext, I need to condense these points into a concise summary that captures the essence without getting bogged down in details.\n\nI should start by stating Newton's view